In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from lxml import etree
import requests

In [85]:
urls = ['https://www.marketwatch.com/investing/stock/sony?mod=mw_quote_recentlyviewed',
       'https://www.marketwatch.com/investing/stock/ibm?mod=mw_quote_recentlyviewed',
       'https://www.marketwatch.com/investing/stock/dell?mod=search_symbol',
       'https://www.marketwatch.com/investing/stock/amd?mod=search_symbol',
        'https://www.marketwatch.com/investing/stock/intc?mod=search_symbol',
       'https://www.marketwatch.com/investing/stock/nvda?mod=search_symbol',
       'https://www.marketwatch.com/investing/stock/hpq?mod=search_symbol',
       'https://www.marketwatch.com/investing/stock/aapl?mod=search_symbol']


In [86]:
company_ = []
open_value_ = []
close_value_ = []
day_range_lower_ = []
day_range_higher_ = []
week_52_range_lower_ = []
week_52_range_higher_ = []
text_ratings_ = []

In [87]:
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    company = re.findall('\/[A-Za-z]+\?', url)
    chars = ['?','/', '[',']', '\'']
    for i in chars:
         company = str(company).replace(i, '')
    company_.append(company)
    
    try:      
        open_value = dom.xpath('/html/body/div[3]/div[2]/div[3]/div/div[2]/h2/bg-quote')[0].text
        open_value_.append(open_value)
    except:
        open_value_.append(np.nan)
        
    try: 
        close_value = dom.xpath('/html/body/div[3]/div[2]/div[3]/div/div[3]/table/tbody/tr/td')[0].text
        close_value_.append(close_value)
    except:
        close_value_.append(np.nan)
        
    try:
        day_range_lower = dom.xpath('/html/body/div[3]/div[2]/div[5]/mw-rangebar[2]/div[1]/span[1]')[0].text
        day_range_lower_.append(day_range_lower)  
    except:
        day_range_lower_.append(np.nan)
    
    try:
        day_range_higher = dom.xpath('/html/body/div[3]/div[2]/div[5]/mw-rangebar[2]/div[1]/span[3]')[0].text
        day_range_higher_.append(day_range_higher)  
    except:
        day_range_higher_.append(np.nan)
        
    try:
        week_52_range_lower = soup.find(class_ = 'element element--range range--yearly').find(class_ = 'primary').text
        week_52_range_lower_.append(week_52_range_lower)
    except:
        week_52_range_lower_.append(np.nan)
        
    try:
        week_52_range_higher = soup.find(class_ = 'element element--range range--yearly').findAll(class_ = 'primary')[1].text    
        week_52_range_higher_.append(week_52_range_higher)
    except:
        week_52_range_higher_.append(np.nan)
        
    try:
        analyst_ratings_list = soup.findAll(class_ = 'analyst__option')
        text_ratings = np.nan
        for element in analyst_ratings_list:
            string_element = str(element)
            if 'active' in string_element:
                text_ratings = element.text
                text_ratings_.append(text_ratings)
    except:
        text_ratings_.append(np.nan)

### Create DataFrame

In [88]:
df = pd.DataFrame({'Company': company_,'Open value': open_value_, 'Close value': close_value_, 
        'Day range lower': day_range_lower_, 'Day range higher': day_range_higher_,
        '52 Week range lower': week_52_range_lower_,
        '52 Week range higher': week_52_range_higher_, 
        'Analyst rating': text_ratings_})
df

,Company,Open value,Close value,Day range lower,Day range higher,52 Week range lower,52 Week range higher,Analyst rating
0,sony,75.12,$74.59,74.41,75.15,61.72,133.75,Buy
1,ibm,138.48,$136.96,136.51,138.70,114.56,144.73,Hold
2,dell,39.81,$39.00,38.94,39.83,32.90,61.54,Over
3,amd,62.63,$62.19,61.40,63.53,54.57,164.46,Over
4,intc,28.23,$28.20,27.89,28.37,24.59,56.28,Hold
5,nvda,142.75,$141.56,139.54,142.99,108.13,346.47,Over
6,hpq,28.80,$28.29,28.27,28.80,24.08,41.47,Hold
7,aapl,138.25,$138.38,135.67,138.33,129.04,182.94,Over


In [89]:
df.to_excel('stock_market.xlsx',index = False)